In [1]:
import numpy as np
import pandas as pd
import math
import random
import time


# Change dataset file Here

In [2]:
# Load the dataset
file_path = 'vopd.csv'
df = pd.read_csv(file_path)



In [3]:
# Define CoreGraph class
class CoreGraph:
    def __init__(self, cores, edges, bandwidths):
        self.cores = cores
        self.edges = edges
        self.bandwidths = bandwidths

    def get_bandwidth(self, core1, core2):
        try:
            index = self.edges.index((core1, core2))
            return self.bandwidths[index]
        except ValueError:
            return 0

# Define helper function to extract core graph information from the dataset
def initialize_core_graph(df):
    source_column = 'source'
    target_column = 'target'
    bandwidth_column = 'weight'

    cores = list(set(df[source_column]).union(set(df[target_column])))
    edges = list(zip(df[source_column], df[target_column]))
    bandwidths = list(df[bandwidth_column])

    core_graph = CoreGraph(cores, edges, bandwidths)
    return core_graph

# Initialize CoreGraph
core_graph = initialize_core_graph(df)

# Display the core graph details
print("Cores:", core_graph.cores)
print("Edges:", core_graph.edges)
print("Bandwidths:", core_graph.bandwidths)

Cores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Edges: [(0, 1), (1, 2), (2, 3), (3, 4), (3, 15), (4, 5), (4, 15), (5, 6), (5, 8), (5, 11), (6, 7), (7, 8), (7, 9), (8, 9), (10, 11), (10, 14), (11, 12), (12, 13), (12, 14), (13, 14)]
Bandwidths: [70, 362, 362, 362, 49, 357, 27, 353, 16, 16, 300, 313, 500, 313, 16, 16, 16, 157, 16, 16]


In [4]:
# Define helper functions to calculate coordinates and hops
def get_coordinates(router_id, X, Y):
    """Convert router ID to (x, y) coordinates."""
    return (router_id // X, router_id % X)

def get_hops(router1, router2):
    """Calculate the Manhattan distance (hops) between two routers."""
    x1, y1 = router1
    x2, y2 = router2
    return abs(x1 - x2) + abs(y1 - y2)

In [5]:

def calculate_grid_dimensions(num_cores):
    """Calculate the grid dimensions (X, Y) for the given number of cores."""
    X = int(math.ceil(math.sqrt(num_cores)))
    Y = int(math.ceil(num_cores / X))
    return X, Y

# Calculate grid dimensions based on the highest core number in the dataset
def calculate_grid_size(df):
    source_column = 'source'
    target_column = 'target'
    
    highest_number = max(df[source_column].max(), df[target_column].max())
    num_cores = highest_number + 1  # Core numbers are 0-based, so add 1
    
    X, Y = calculate_grid_dimensions(num_cores)
    
    return highest_number, num_cores, X, Y

# Calculate the grid dimensions
highest_number, num_cores, X, Y = calculate_grid_size(df)

# Print grid dimensions
print(f"Highest Core Number: {highest_number}")
print(f"Number of Cores: {num_cores}")
print(f"Grid Dimensions: {X} x {Y}")


Highest Core Number: 15
Number of Cores: 16
Grid Dimensions: 4 x 4


In [6]:
def get_coordinates(router_id, X, Y):
    """Convert router ID to (x, y) coordinates."""
    return (router_id // X, router_id % X)

def verify_router_coordinates(mapping, X, Y):
    for router, core in enumerate(mapping):
        router_coords = get_coordinates(router, X, Y)
        print(f"Core: {core}, Router ID: {router}, Router Coordinates: {router_coords}")

# Test with a sample mapping array
sample_mapping_array = np.array([5, 6, 3, 4, 7, 8, 0, 1, 2])

# Verify router coordinates
verify_router_coordinates(sample_mapping_array, X, Y)


Core: 5, Router ID: 0, Router Coordinates: (0, 0)
Core: 6, Router ID: 1, Router Coordinates: (0, 1)
Core: 3, Router ID: 2, Router Coordinates: (0, 2)
Core: 4, Router ID: 3, Router Coordinates: (0, 3)
Core: 7, Router ID: 4, Router Coordinates: (1, 0)
Core: 8, Router ID: 5, Router Coordinates: (1, 1)
Core: 0, Router ID: 6, Router Coordinates: (1, 2)
Core: 1, Router ID: 7, Router Coordinates: (1, 3)
Core: 2, Router ID: 8, Router Coordinates: (2, 0)


In [7]:
def compute_communication_cost(mapping, core_graph, X, Y):
    """Compute the total communication cost for a given mapping."""
    cost = 0
    for (core1, core2) in core_graph.edges:
        # Get the router indices from the mapping array
        router_id1 = np.where(mapping == core1)[0][0]
        router_id2 = np.where(mapping == core2)[0][0]
        # Get the router coordinates
        router1 = get_coordinates(router_id1, X, Y)
        router2 = get_coordinates(router_id2, X, Y)
        # Calculate hops between the routers
        hops = get_hops(router1, router2)
        # Get the bandwidth for the edge
        bandwidth = core_graph.get_bandwidth(core1, core2)
        # Calculate the cost as hops * bandwidth
        cost += hops * bandwidth
    return cost

def evaluate_population(population, core_graph, X, Y):
    fitness_scores = []
    for mapping in population:
        cost = compute_communication_cost(mapping, core_graph, X, Y)
        fitness_scores.append(cost)
    return fitness_scores

# # Example usage
# population_ex = [np.array([0, 1, 2, 3, 4, 5, 6, 7, 8]), np.array([5, 6, 3, 4, 7, 8, 0, 1, 2])]
# fitness_scores_ex = evaluate_population(population_ex, core_graph, X, Y)

# # Print fitness scores
# for i, score in enumerate(fitness_scores_ex):
#     print(f"Individual {i}: Communication Cost = {score}")


In [8]:
import numpy as np

def initialize_population(pop_size, num_routers):
    population = []
    for _ in range(pop_size):
        mapping = np.random.permutation(num_routers)
        population.append(mapping)
    return population


In [9]:
# Define the population size and number of routers
pop_size = 50
num_routers = X*Y

# Initialize the population
population = initialize_population(pop_size, num_routers)

# Print the initialized population
for i, individual in enumerate(population):
    print(f"Individual {i}: {individual}")


Individual 0: [15 12 13  3  9 10  7 14  1  8  0  4  2  5  6 11]
Individual 1: [ 9 10  7  8  2 14  4  1 13  5 15  3  6 12 11  0]
Individual 2: [12 11  1 10  9  5 15  7 13 14  6  8  2  3  0  4]
Individual 3: [ 1 10  2  5 13 11  4  3 12 15 14  9  8  6  0  7]
Individual 4: [ 1  5  8 13 14 12  3  6 15  4  7  0 11  2 10  9]
Individual 5: [ 5  3 13  4  9  8 12 11  0  2 10  1  7 14 15  6]
Individual 6: [ 0  5  9  8 13 10  4  7 11 12 15  2  6  1  3 14]
Individual 7: [15  1  9  2  4 10 11 14 12  0 13  7  6  8  3  5]
Individual 8: [12  7  5 13 15 11  2 14  6  4  8 10  9  0  1  3]
Individual 9: [ 2 10  6  8  1 15  3 11  4 14 13  0  9  7  5 12]
Individual 10: [ 8  2  4  5 11 15  3  1  0 13  9  7  6 12 14 10]
Individual 11: [14  3 13  2  5 11  0 10  6 15 12  4  1  9  7  8]
Individual 12: [ 3 14 10  2  8  1  4 13  7  9  5 12 11  6 15  0]
Individual 13: [ 9  2  3 12  8 15 11  4  7  5  0 13  1 10  6 14]
Individual 14: [15 10 14  7  0  1  6  5  2  4  3 12  8 11  9 13]
Individual 15: [ 0  3 14 10  9  4  

In [10]:
num_routers

16

In [11]:
fitness_scores = evaluate_population(population, core_graph, X, Y)

In [12]:
# Print fitness scores
for i, score in enumerate(fitness_scores):
    print(f"Individual {i}: Communication Cost = {score}")

Individual 0: Communication Cost = 8417
Individual 1: Communication Cost = 9216
Individual 2: Communication Cost = 9687
Individual 3: Communication Cost = 8608
Individual 4: Communication Cost = 9808
Individual 5: Communication Cost = 9472
Individual 6: Communication Cost = 8776
Individual 7: Communication Cost = 12353
Individual 8: Communication Cost = 11237
Individual 9: Communication Cost = 10792
Individual 10: Communication Cost = 10008
Individual 11: Communication Cost = 9530
Individual 12: Communication Cost = 7464
Individual 13: Communication Cost = 7753
Individual 14: Communication Cost = 9210
Individual 15: Communication Cost = 8879
Individual 16: Communication Cost = 10639
Individual 17: Communication Cost = 11827
Individual 18: Communication Cost = 9059
Individual 19: Communication Cost = 10667
Individual 20: Communication Cost = 9493
Individual 21: Communication Cost = 12102
Individual 22: Communication Cost = 12023
Individual 23: Communication Cost = 8901
Individual 24: Co

In [13]:


def tournament_selection(population, fitness_scores, tournament_size):
    """Perform tournament selection to choose one individual."""
    tournament = random.sample(range(len(population)), tournament_size)
    best_individual = min(tournament, key=lambda i: fitness_scores[i])
    return population[best_individual]


In [14]:
# Define the tournament size
tournament_size = 3

# Perform tournament selection
selected_individual = tournament_selection(population, fitness_scores, tournament_size)

# Print the selected individual
print("Selected Individual:", selected_individual)


Selected Individual: [ 2  6  5 10 15  9 11 13  3  7  8  1 14  0  4 12]


In [15]:
selected_individual

array([ 2,  6,  5, 10, 15,  9, 11, 13,  3,  7,  8,  1, 14,  0,  4, 12])

In [16]:
compute_communication_cost(selected_individual, core_graph, X, Y)

8116

In [17]:
def crossover(parent1, parent2):
    size = len(parent1)
    crossover_point = np.random.randint(1, size - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    # Resolve duplicates in child1
    unique_child1, indices1 = np.unique(child1, return_index=True)
    missing1 = np.setdiff1d(np.arange(size), unique_child1)
    child1 = np.concatenate((unique_child1, missing1))

    # Resolve duplicates in child2
    unique_child2, indices2 = np.unique(child2, return_index=True)
    missing2 = np.setdiff1d(np.arange(size), unique_child2)
    child2 = np.concatenate((unique_child2, missing2))

    return child1, child2


In [18]:
# Example parent solutions
parent1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
parent2 = np.array([8, 7, 6, 5, 4, 3, 2, 1, 0])

# Perform crossover
child1, child2 = crossover(parent1, parent2)

# Print the resulting children
print("Child 1:", child1)
print("Child 2:", child2)


Child 1: [0 1 2 3 4 5 6 7 8]
Child 2: [2 3 4 5 6 7 8 0 1]


In [19]:
# compute_communication_cost(child2, core_graph, X, Y)

In [20]:
def mutate(mapping, mutation_rate=0.1):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(mapping), size=2, replace=False)
        mapping[idx1], mapping[idx2] = mapping[idx2], mapping[idx1]
    return mapping


In [21]:
# Example mapping
mapping = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])

# Perform mutation
mutated_mapping = mutate(mapping, mutation_rate=1)

# Print the mutated mapping
print("Mutated Mapping:", mutated_mapping)


Mutated Mapping: [0 1 2 3 4 7 6 5 8]


In [22]:

def run_genetic_algorithm(pop_size, grid_size, core_graph, X, Y, generations, tournament_size, mutation_rate):
    """Run the genetic algorithm with detailed debugging output and track best solutions per generation."""
    # Initialize population
    population = initialize_population(pop_size, grid_size)
    
    all_mappings = []
    all_costs = []
    best_mappings_per_gen = []
    best_costs_per_gen = []
    runtimes_per_gen = []
    
    for generation in range(generations):
        print(f"\nGeneration {generation}")
        
        # Record start time
        start_time = time.time()
        
        # Evaluate population
        fitness_scores = evaluate_population(population, core_graph, X, Y)
        
        # Store mappings and costs
        all_mappings.append(population.copy())
        all_costs.append(fitness_scores.copy())
        
        # Select the next generation
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, fitness_scores, tournament_size)
            parent2 = tournament_selection(population, fitness_scores, tournament_size)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.append(child1)
            new_population.append(child2)
        
        # Evaluate new population
        new_fitness_scores = evaluate_population(new_population, core_graph, X, Y)
        
        # Combine old and new populations
        combined_population = population + new_population
        combined_fitness_scores = fitness_scores + new_fitness_scores
        
        # Select the best individuals to form the next generation
        sorted_indices = np.argsort(combined_fitness_scores)
        population = [combined_population[i] for i in sorted_indices[:pop_size]]
        fitness_scores = [combined_fitness_scores[i] for i in sorted_indices[:pop_size]]
        
        best_cost = min(fitness_scores)
        best_mapping = population[np.argmin(fitness_scores)]
        
        # Store best mapping and cost for this generation
        best_mappings_per_gen.append(best_mapping)
        best_costs_per_gen.append(best_cost)
        
        print(f"Best Cost This Generation: {best_cost}")
        
        # Record end time and calculate duration
        end_time = time.time()
        generation_runtime = end_time - start_time
        runtimes_per_gen.append(generation_runtime)
#         print(f"Runtime This Generation: {generation_runtime} seconds")
    
    # Final evaluation
    fitness_scores = evaluate_population(population, core_graph, X, Y)
    best_mapping = population[np.argmin(fitness_scores)]
    best_cost = min(fitness_scores)
    
    best_mappings_per_gen.append(best_mapping)
    best_costs_per_gen.append(best_cost)
    
    return best_mapping, best_cost, best_mappings_per_gen, best_costs_per_gen, runtimes_per_gen


# For run multiple trial

In [23]:
# Example usage
highest_number, num_cores, X, Y = calculate_grid_size(df)
core_graph = initialize_core_graph(df)
pop_size = 200
grid_size = X * Y
generations = 1000
tournament_size = 4
mutation_rate = 0.5

# Variables to store results
best_costs = []
total_runtimes = []
all_best_costs_per_gen = []
convergence_points = []

# Run the genetic algorithm 10 times and record results
for i in range(10):
    print(f"Run {i + 1}/10")
    
    
    best_mapping, best_cost, best_mappings_per_gen, best_costs_per_gen, runtimes_per_gen = run_genetic_algorithm(
        pop_size, grid_size, core_graph, X, Y, generations, tournament_size, mutation_rate
    )
    

    total_runtime = sum(runtimes_per_gen)
    
    best_costs.append(best_cost)
    total_runtimes.append(total_runtime)
    # Find convergence point
    min_index = best_costs_per_gen.index(min(best_costs_per_gen))
    convergence_points.append(min_index)
    all_best_costs_per_gen.append(best_costs_per_gen.copy())

Run 1/10

Generation 0
Best Cost This Generation: 6207

Generation 1
Best Cost This Generation: 5047

Generation 2
Best Cost This Generation: 5047

Generation 3
Best Cost This Generation: 5047

Generation 4
Best Cost This Generation: 5047

Generation 5
Best Cost This Generation: 5031

Generation 6
Best Cost This Generation: 5031

Generation 7
Best Cost This Generation: 5031

Generation 8
Best Cost This Generation: 5031

Generation 9
Best Cost This Generation: 5031

Generation 10
Best Cost This Generation: 5031

Generation 11
Best Cost This Generation: 5031

Generation 12
Best Cost This Generation: 5031

Generation 13
Best Cost This Generation: 5031

Generation 14
Best Cost This Generation: 5031

Generation 15
Best Cost This Generation: 5031

Generation 16
Best Cost This Generation: 5031

Generation 17
Best Cost This Generation: 5031

Generation 18
Best Cost This Generation: 5031

Generation 19
Best Cost This Generation: 5031

Generation 20
Best Cost This Generation: 5031

Generation 21

Best Cost This Generation: 5031

Generation 176
Best Cost This Generation: 5031

Generation 177
Best Cost This Generation: 5031

Generation 178
Best Cost This Generation: 5031

Generation 179
Best Cost This Generation: 5031

Generation 180
Best Cost This Generation: 5031

Generation 181
Best Cost This Generation: 5031

Generation 182
Best Cost This Generation: 5031

Generation 183
Best Cost This Generation: 5031

Generation 184
Best Cost This Generation: 5031

Generation 185
Best Cost This Generation: 5031

Generation 186
Best Cost This Generation: 5031

Generation 187
Best Cost This Generation: 5031

Generation 188
Best Cost This Generation: 5031

Generation 189
Best Cost This Generation: 5031

Generation 190
Best Cost This Generation: 5031

Generation 191
Best Cost This Generation: 5031

Generation 192
Best Cost This Generation: 5031

Generation 193
Best Cost This Generation: 5031

Generation 194
Best Cost This Generation: 5031

Generation 195
Best Cost This Generation: 5031

Generat

Best Cost This Generation: 5031

Generation 350
Best Cost This Generation: 5031

Generation 351
Best Cost This Generation: 5031

Generation 352
Best Cost This Generation: 5031

Generation 353
Best Cost This Generation: 5031

Generation 354
Best Cost This Generation: 5031

Generation 355
Best Cost This Generation: 5031

Generation 356
Best Cost This Generation: 5031

Generation 357
Best Cost This Generation: 5031

Generation 358
Best Cost This Generation: 5031

Generation 359
Best Cost This Generation: 5031

Generation 360
Best Cost This Generation: 5031

Generation 361
Best Cost This Generation: 5031

Generation 362
Best Cost This Generation: 5031

Generation 363
Best Cost This Generation: 5031

Generation 364
Best Cost This Generation: 5031

Generation 365
Best Cost This Generation: 5031

Generation 366
Best Cost This Generation: 5031

Generation 367
Best Cost This Generation: 5031

Generation 368
Best Cost This Generation: 5031

Generation 369
Best Cost This Generation: 5031

Generat

Best Cost This Generation: 5031

Generation 521
Best Cost This Generation: 5031

Generation 522
Best Cost This Generation: 5031

Generation 523
Best Cost This Generation: 5031

Generation 524
Best Cost This Generation: 5031

Generation 525
Best Cost This Generation: 5031

Generation 526
Best Cost This Generation: 5031

Generation 527
Best Cost This Generation: 5031

Generation 528
Best Cost This Generation: 5031

Generation 529
Best Cost This Generation: 5031

Generation 530
Best Cost This Generation: 5031

Generation 531
Best Cost This Generation: 5031

Generation 532
Best Cost This Generation: 5031

Generation 533
Best Cost This Generation: 5031

Generation 534
Best Cost This Generation: 5031

Generation 535
Best Cost This Generation: 5031

Generation 536
Best Cost This Generation: 5031

Generation 537
Best Cost This Generation: 5031

Generation 538
Best Cost This Generation: 5031

Generation 539
Best Cost This Generation: 5031

Generation 540
Best Cost This Generation: 5031

Generat

Best Cost This Generation: 5031

Generation 695
Best Cost This Generation: 5031

Generation 696
Best Cost This Generation: 5031

Generation 697
Best Cost This Generation: 5031

Generation 698
Best Cost This Generation: 5031

Generation 699
Best Cost This Generation: 5031

Generation 700
Best Cost This Generation: 5031

Generation 701
Best Cost This Generation: 5031

Generation 702
Best Cost This Generation: 5031

Generation 703
Best Cost This Generation: 5031

Generation 704
Best Cost This Generation: 5031

Generation 705
Best Cost This Generation: 5031

Generation 706
Best Cost This Generation: 5031

Generation 707
Best Cost This Generation: 5031

Generation 708
Best Cost This Generation: 5031

Generation 709
Best Cost This Generation: 5031

Generation 710
Best Cost This Generation: 5031

Generation 711
Best Cost This Generation: 5031

Generation 712
Best Cost This Generation: 5031

Generation 713
Best Cost This Generation: 5031

Generation 714
Best Cost This Generation: 5031

Generat

...

Best Cost This Generation: 5031

Generation 147
Best Cost This Generation: 5031

Generation 148
Best Cost This Generation: 5031

Generation 149
Best Cost This Generation: 5031

Generation 150
Best Cost This Generation: 5031

Generation 151
Best Cost This Generation: 5031

Generation 152
Best Cost This Generation: 5031

Generation 153
Best Cost This Generation: 5031

Generation 154
Best Cost This Generation: 5031

Generation 155
Best Cost This Generation: 5031

Generation 156
Best Cost This Generation: 4955

Generation 157
Best Cost This Generation: 4955

Generation 158
Best Cost This Generation: 4955

Generation 159
Best Cost This Generation: 4955

Generation 160
Best Cost This Generation: 4955

Generation 161
Best Cost This Generation: 4955

Generation 162
Best Cost This Generation: 4955

Generation 163
Best Cost This Generation: 4955

Generation 164
Best Cost This Generation: 4955

Generation 165
Best Cost This Generation: 4955

Generation 166
Best Cost This Generation: 4955

Generat

Best Cost This Generation: 4955

Generation 320
Best Cost This Generation: 4955

Generation 321
Best Cost This Generation: 4955

Generation 322
Best Cost This Generation: 4955

Generation 323
Best Cost This Generation: 4955

Generation 324
Best Cost This Generation: 4955

Generation 325
Best Cost This Generation: 4955

Generation 326
Best Cost This Generation: 4955

Generation 327
Best Cost This Generation: 4955

Generation 328
Best Cost This Generation: 4955

Generation 329
Best Cost This Generation: 4955

Generation 330
Best Cost This Generation: 4955

Generation 331
Best Cost This Generation: 4955

Generation 332
Best Cost This Generation: 4955

Generation 333
Best Cost This Generation: 4955

Generation 334
Best Cost This Generation: 4955

Generation 335
Best Cost This Generation: 4955

Generation 336
Best Cost This Generation: 4955

Generation 337
Best Cost This Generation: 4955

Generation 338
Best Cost This Generation: 4955

Generation 339
Best Cost This Generation: 4955

Generat

Best Cost This Generation: 4955

Generation 493
Best Cost This Generation: 4955

Generation 494
Best Cost This Generation: 4955

Generation 495
Best Cost This Generation: 4955

Generation 496
Best Cost This Generation: 4955

Generation 497
Best Cost This Generation: 4955

Generation 498
Best Cost This Generation: 4955

Generation 499
Best Cost This Generation: 4955

Generation 500
Best Cost This Generation: 4955

Generation 501
Best Cost This Generation: 4955

Generation 502
Best Cost This Generation: 4955

Generation 503
Best Cost This Generation: 4955

Generation 504
Best Cost This Generation: 4955

Generation 505
Best Cost This Generation: 4955

Generation 506
Best Cost This Generation: 4955

Generation 507
Best Cost This Generation: 4955

Generation 508
Best Cost This Generation: 4955

Generation 509
Best Cost This Generation: 4955

Generation 510
Best Cost This Generation: 4955

Generation 511
Best Cost This Generation: 4955

Generation 512
Best Cost This Generation: 4955

Generat

Best Cost This Generation: 4955

Generation 666
Best Cost This Generation: 4955

Generation 667
Best Cost This Generation: 4955

Generation 668
Best Cost This Generation: 4955

Generation 669
Best Cost This Generation: 4955

Generation 670
Best Cost This Generation: 4955

Generation 671
Best Cost This Generation: 4955

Generation 672
Best Cost This Generation: 4955

Generation 673
Best Cost This Generation: 4955

Generation 674
Best Cost This Generation: 4955

Generation 675
Best Cost This Generation: 4955

Generation 676
Best Cost This Generation: 4955

Generation 677
Best Cost This Generation: 4955

Generation 678
Best Cost This Generation: 4955

Generation 679
Best Cost This Generation: 4955

Generation 680
Best Cost This Generation: 4955

Generation 681
Best Cost This Generation: 4955

Generation 682
Best Cost This Generation: 4955

Generation 683
Best Cost This Generation: 4955

Generation 684
Best Cost This Generation: 4955

Generation 685
Best Cost This Generation: 4955

Generat

Best Cost This Generation: 4955

Generation 839
Best Cost This Generation: 4955

Generation 840
Best Cost This Generation: 4955

Generation 841
Best Cost This Generation: 4955

Generation 842
Best Cost This Generation: 4955

Generation 843
Best Cost This Generation: 4955

Generation 844
Best Cost This Generation: 4955

Generation 845
Best Cost This Generation: 4955

Generation 846
Best Cost This Generation: 4955

Generation 847
Best Cost This Generation: 4955

Generation 848
Best Cost This Generation: 4955

Generation 849
Best Cost This Generation: 4955

Generation 850
Best Cost This Generation: 4955

Generation 851
Best Cost This Generation: 4955

Generation 852
Best Cost This Generation: 4955

Generation 853
Best Cost This Generation: 4955

Generation 854
Best Cost This Generation: 4955

Generation 855
Best Cost This Generation: 4955

Generation 856
Best Cost This Generation: 4955

Generation 857
Best Cost This Generation: 4955

Generation 858
Best Cost This Generation: 4955

Generat

In [24]:
# Create a DataFrame
results_df = pd.DataFrame({
    'Best Cost': best_costs,
    'Total Runtime': total_runtimes,
    'Convergence Point': convergence_points
})

In [25]:
results_df

,Best Cost,Total Runtime,Convergence Point
0,5031,78.961110,5
1,5031,80.676317,3
2,5031,80.681510,13
3,4923,79.762233,45
4,5031,80.639773,4
5,5031,81.839948,7
6,5031,81.648982,4
7,5031,81.787314,4
8,4923,81.016082,504
9,4955,81.232023,156


In [26]:
# Convert the list of lists to a DataFrame
df_all_cost = pd.DataFrame(all_best_costs_per_gen).transpose()
# Add headers
df_all_cost.columns = [f'Run {i+1}' for i in range(df_all_cost.shape[1])]

In [27]:
df_all_cost[0:5]

,Run 1,Run 2,Run 3,Run 4,Run 5,Run 6,Run 7,Run 8,Run 9,Run 10
0,6207,5686,5957,6068,6012,6183,5785,5909,5529,5348
1,5047,5386,5957,5386,5834,5736,5785,5909,5502,5348
2,5047,5386,5957,5386,5690,5386,5716,5909,5502,5031
3,5047,5031,5386,5031,5690,5386,5716,5386,5502,5031
4,5047,5031,5386,5031,5031,5386,5031,5031,5031,5031


In [40]:
df_all_cost.shape

(1001, 10)

# Export to excel

In [41]:
results_df.to_excel('Summary_Org_name.xlsx', index=False)

In [42]:
df_all_cost.to_excel('rawdata_Org_name.xlsx', index=False)

# For single run

In [29]:
# # Example usage
# highest_number, num_cores, X, Y = calculate_grid_size(df)
# core_graph = initialize_core_graph(df)
# pop_size = 200
# grid_size = X * Y
# generations = 1000
# tournament_size = 4
# mutation_rate = 0.5

# best_mapping, best_cost, best_mappings_per_gen, best_costs_per_gen,runtimes_per_gen = run_genetic_algorithm(pop_size, grid_size, core_graph, X, Y, generations, tournament_size, mutation_rate)

# # Print the best mapping and cost
# print("Best Mapping:", best_mapping)
# print("Best Cost:", best_cost)
# # print("Best Mappings per Generation:", best_mappings_per_gen)
# # print("Best Costs per Generation:", best_costs_per_gen)



In [30]:
min_index = best_costs_per_gen.index(min(best_costs_per_gen))

In [31]:
min_index

156

In [32]:
best_mappings_per_gen[min_index]

array([ 0,  1,  2,  3,  7,  5,  6,  4,  8,  9, 10, 15, 11, 12, 13, 14])

In [33]:
best_cost

4955

In [34]:
best_mapping

array([ 0,  1,  2,  3,  7,  5,  6,  4,  8,  9, 10, 15, 11, 12, 13, 14])

In [35]:
compute_communication_cost(best_mapping, core_graph, X, Y)

4955

In [36]:
best_costs_per_gen[min_index]

4955

In [37]:
 sum(runtimes_per_gen)

81.23202300071716

In [38]:
 sum(runtimes_per_gen)/len(runtimes_per_gen)

0.08123202300071716

In [39]:
len(runtimes_per_gen)

1000